# Model Comparison on Summarization Tasks
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Evals/Compare_Evals.ipynb)

<img src="../images/compare_eval.png" width="750">

## Introduction

This **exercise notebook** guides you through comparing the quality of a base model, fine-tuned model, and proprietary model using Together AI's Evaluations service on the HelpSteer3 conversational dataset.

**You will learn to:**
- Fine-tune a base model on the HelpSteer3 dataset
- Use external models (OpenAI) alongside Together AI models
- Create an LLM-as-a-Judge evaluation pipeline
- Compare model outputs head-to-head

The full list of supported models can be found [here](https://docs.together.ai/docs/evaluations-supported-models).


**Concepts Covered:**
- **LLM-as-a-Judge**: Using a strong model to evaluate and compare outputs from other models
- **Compare Evaluation**: Head-to-head comparison between multiple models
- **Fine-tuning**: Training a base model on domain-specific data
- **External Model Integration**: Using models from different providers alongside Together AI


## Installation and Setup

To set up the environment:
1. Navigate to the same folder as this notebook
2. Run the installation script: `bash install.sh`
   - This will create a virtual environment called `env_cookbook_evals`
   - It will install all dependencies from `requirements.txt`
3. Activate the environment: `source env_cookbook_evals/bin/activate`
4. Put your API TOKENS into .env if you want to use load_dotenv
4. You're ready to run this notebook!

In [ ]:
import json
import os
import time

from datasets import load_dataset
from dotenv import load_dotenv
from jinja2 import Template
from together import Together
from transformers import AutoTokenizer

load_dotenv()

TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
WANDB_API_KEY = os.getenv("WANDB_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


### Model Configurations

We will use:
- **Judge Model**: `moonshotai/Kimi-K2-Instruct-0905`
- **Proprietary Model**: `openai/gpt-5-mini`

For the OSS model, we have 2 options:

1. **Quick Demo** (`meta-llama/Meta-Llama-3.1-8B-Instruct`):
   - Fine-tunes in ~15 minutes
   - Can be used with LoRA serverless
   - Good for showcasing functionality

2. **More Realistic for Quality** (`Qwen/Qwen3-Next-80B-A3B-Instruct`):
   - Fine-tunes in ~2 hours
   - More realistic setup for quality evaluation
   - Requires a dedicated endpoint to run evaluation

In [ ]:

JUDGE_MODEL = "moonshotai/Kimi-K2-Instruct-0905"
PROPRIETARY_BASE_MODEL = "openai/gpt-5-mini"

BASE_OSS_MODEL_FOR_INFERENCE = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
BASE_OSS_MODEL_FOR_FT = "meta-llama/Meta-Llama-3.1-8B-Instruct-Reference"

# !!! Uncomment this for more realistic, but slow (several hours) setup.
# BASE_OSS_MODEL_FOR_INFERENCE = "Qwen/Qwen3-Next-80B-A3B-Instruct"
# BASE_OSS_MODEL_FOR_FT = "Qwen/Qwen3-Next-80B-A3B-Instruct"

## 📊 Understanding the HelpSteer3 Edit Dataset

The HelpSteer3 Edit dataset contains conversational contexts paired with multiple response options that can be compared and evaluated.

**Key Column for Our Setup:**
- We use the `edited_response` column, which represents the ideal "golden response"
- This can be human-provided or generated by a strong proprietary model (e.g., GPT-4)

**Evaluation Approach:**
- Compare how different models respond to the same prompts
- Assess which model produces higher-quality outputs

In [ ]:
hs3_edit = load_dataset("nvidia/HelpSteer3", "edit")

In [ ]:
# Print all unique domain values
print("Unique domains in the dataset:")
print(hs3_edit['train'].unique('domain'))


In [ ]:
# Print a sample to understand the structure
sample = hs3_edit['train'][0]
print("Sample context:")
print(sample['context'])
print("\nSample edited_response:")
print(sample['edited_response'])

## 🔧 SFT Fine-tuning with HelpSteer3 Edit Dataset

We'll use the `context` and `edited_response` columns to create training data for Supervised Fine-Tuning (SFT). The context already contains conversation messages, and we'll append the edited_response as the final assistant message.


In [ ]:
together_client = Together(api_key=TOGETHER_API_KEY)

In [ ]:
def map_to_sft_format(row):
    """Convert HelpSteer3 row to SFT chat format by appending edited_response to context.
    
    Args:
        row: Dict with 'context' (list of messages) and 'edited_response' (string)
    
    Returns:
        Dict with 'messages' key containing full conversation including the golden response
    """
    # TODO: Exercise 1 - Implement this function
    # Hint 1: Copy the context messages to a new list (don't modify original)
    # Hint 2: Append a new assistant message with the edited_response content
    # Hint 3: Return a dict with key 'messages' containing the complete conversation
    
    raise NotImplementedError("Implement map_to_sft_format")

# Apply transformation to the dataset
train_sft = hs3_edit['train'].map(map_to_sft_format, remove_columns=hs3_edit['train'].column_names)
print(f"Transformed dataset size: {len(train_sft)}")
print(f"Sample messages count: {len(train_sft[0]['messages'])}")


In [ ]:
# Validate the dataset format
assert 'messages' in train_sft.column_names, "Dataset must contain 'messages' column"
assert len(train_sft) > 0, "Dataset must not be empty"
assert isinstance(train_sft[0]['messages'], list), "Messages must be a list"
assert all('role' in msg and 'content' in msg for msg in train_sft[0]['messages']), "Each message must have 'role' and 'content'"
print("✓ Dataset format validation passed")

In [ ]:
train_sft[0]['messages']

In [ ]:
# Save to JSONL file
SFT_TRAIN_FILE = "helpsteer3_sft_train.jsonl"
train_sft.to_json(SFT_TRAIN_FILE)
print(f"Saved training data to {SFT_TRAIN_FILE}")


In [ ]:
# Upload file to Together AI
train_file_resp = together_client.files.upload(SFT_TRAIN_FILE, purpose='fine-tune', check=True)
print(f"Uploaded file ID: {train_file_resp.id}")


### Launch Fine-tuning Job

Configure and start the SFT fine-tuning job using the uploaded HelpSteer3 data.


### Training Takes about 15 mins, so we can start it, and then proceed to Evaluations part without waiting.

In [ ]:
# TODO: Exercise 2 - Create a fine-tuning job
# Fill in the missing parameters below
# Docs: https://docs.together.ai/docs/fine-tuning-quickstart

ft_resp = together_client.fine_tuning.create(
    training_file=train_file_resp.id,
    model=BASE_OSS_MODEL_FOR_FT,
    # TODO: Set train_on_inputs to False (we only want to train on the assistant response)
    train_on_inputs=None,  # <-- Replace None
    # TODO: Set number of epochs to 1
    n_epochs=None,  # <-- Replace None
    # TODO: Set number of checkpoints to 1
    n_checkpoints=None,  # <-- Replace None
    wandb_api_key=WANDB_API_KEY if WANDB_API_KEY else None,
    # TODO: Enable LoRA fine-tuning (set to True)
    lora=None,  # <-- Replace None
    suffix="helpsteer3-sft",
)

print(f"Fine-tuning job ID: {ft_resp.id}")
print(f"Status: {ft_resp.status}")

### Monitor Fine-tuning Progress


In [ ]:
# Check job status
job_status = together_client.fine_tuning.retrieve(ft_resp.id)
print(f"Status: {job_status.status}")

# List events/logs
for event in together_client.fine_tuning.list_events(id=ft_resp.id).data:
    print(event.message)

print(ft_resp.id)

Now we can move to the evaluations, because fine-tuning will take some time.

## 🔄 Preparing Data for Evaluation

We'll sample 50 random examples from the original test set and prepare them for evaluation. The evaluation will compare:
1. Proprietary model vs Base OSS model - to see how a proprietary model compares to a base open-source model
2. Proprietary model vs Fine-tuned OSS model - to measure if fine-tuning closes the gap with proprietary models

The judge will use the golden answer (edited_response) from the dataset as a reference to determine which model's response is better aligned with the ideal answer.

We need to:
- Apply a chat template to convert the context messages into a formatted prompt string
- Include the golden answer (edited_response) for the judge to use as reference

In [ ]:
# Let's use 50 samples for validation for speed
VALIDATION_SIZE = 50
test_data = hs3_edit['validation'].shuffle(seed=42).select(range(VALIDATION_SIZE))
print(f"Test subset size: {len(test_data)}")

In [ ]:
# Print all unique values of 'domain' column in test_data
# There are 4 domains in the original dataset.
unique_domains = set(test_data['domain'])
print(f"Unique domains: {unique_domains}")

In [ ]:
# Load tokenizer for the base model to apply proper chat template
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

def prepare_eval_data(row):
    """Prepare a single row for evaluation with formatted context and golden answer.
    
    Args:
        row: Dict with 'context' (list of messages) and 'edited_response' (string)
    
    Returns:
        Dict with 'context_formatted' and 'golden_answer' keys
    """
    # TODO: Exercise 3 - Implement this function
    # Step 1: Use tokenizer.apply_chat_template() to format the context
    #         - Pass row['context'] as the messages
    #         - Set tokenize=False to get string output
    #         - Set add_generation_prompt=False
    # Step 2: Return a dict with:
    #         - 'context_formatted': the formatted context string
    #         - 'golden_answer': the edited_response from the row
    
    raise NotImplementedError("Implement prepare_eval_data")

# Transform test data for evaluation
eval_data = [prepare_eval_data(row) for row in test_data]
print(f"Prepared {len(eval_data)} samples for evaluation")

In [ ]:
# Verify the eval_data format
assert len(eval_data) > 0, "eval_data should not be empty"
assert all('context_formatted' in item for item in eval_data), "All items must have 'context_formatted' key"
assert all('golden_answer' in item for item in eval_data), "All items must have 'golden_answer' key"
assert all(isinstance(item['context_formatted'], str) for item in eval_data), "context_formatted must be strings"
assert all(isinstance(item['golden_answer'], str) for item in eval_data), "golden_answer must be strings"
print("✓ Eval data format validated successfully")

In [ ]:
# Save evaluation data to JSONL and upload
EVAL_FILE = "helpsteer3_eval.jsonl"
with open(EVAL_FILE, 'w') as f:
    for eval_item in eval_data:
        json.dump(eval_item, f)
        f.write('\n')

uploaded_eval_file = together_client.files.upload(file=EVAL_FILE, purpose='eval', check=False)
print(f"Uploaded eval file ID: {uploaded_eval_file.id}")

In [ ]:
# Model and judge configuration

# To refer a field from a dataset in templates, use 4 brackets: {{{{field_name}}}}
# This becomes {{field_name}} in the final Jinja template

# TODO: Exercise 4 - Write the judge template
# The judge should evaluate responses based on:
# - Helpfulness, Accuracy, Clarity, Completeness, Safety
# - Alignment with the golden answer (reference response)
# 
# Include the golden_answer field in your template using: {{{{golden_answer}}}}

JUDGE_TEMPLATE = f"""
# TODO: Write your judge template here
# Include evaluation criteria and instructions for the judge
# Make sure to reference the golden answer using {{{{golden_answer}}}}
"""

# Model config for generation from context
generation_system_template = "You are a helpful AI assistant."
# This template tells the model what input to use for generation
input_template = f"{{{{context_formatted}}}}"

In [ ]:
# Test that jinja template works
test_context = "<<This is a test context>>"
test_template = Template(input_template)
rendered = test_template.render(context_formatted=test_context)
assert test_context in rendered, "Jinja template rendering failed"
print(rendered)


In [ ]:
MAX_TOKENS = 8096
TEMPERATURE = 0.7

## 🏃‍♂️ Evaluation 1: Proprietary Model vs Base Model

Compare the proprietary model's output against the base OSS model, with the judge considering alignment with the golden answer.

In [ ]:
# TODO: Exercise 5 - Configure models and create evaluation
# Docs: https://docs.together.ai/docs/ai-evaluations

# Model A: Proprietary model (external - OpenAI)
proprietary_model_config = {
    "model": PROPRIETARY_BASE_MODEL,
    "model_source": "external",  # "external" for non-Together models
    "system_template": generation_system_template,
    "input_template": input_template,
    "max_tokens": MAX_TOKENS,
    "temperature": TEMPERATURE,
    # TODO: Add the external_api_token for OpenAI
    # "external_api_token": ???
}

# Model B: Base OSS model (serverless - Together AI)
base_model_config = {
    "model": BASE_OSS_MODEL_FOR_INFERENCE,
    # TODO: Set model_source to "serverless" for Together AI serverless models
    "model_source": None,  # <-- Replace None
    "system_template": generation_system_template,
    "input_template": input_template,
    "max_tokens": MAX_TOKENS,
    "temperature": TEMPERATURE
}

# TODO: Create the evaluation using together_client.evaluation.create()
# Required parameters:
#   - type: "compare" (for head-to-head comparison)
#   - input_data_file_path: uploaded_eval_file.id
#   - judge_model: JUDGE_MODEL
#   - judge_model_source: "serverless"
#   - judge_system_template: JUDGE_TEMPLATE
#   - model_a: proprietary_model_config
#   - model_b: base_model_config

eval_proprietary_vs_base = None  # <-- Replace with evaluation.create() call

print(f"Eval 1 (Proprietary vs Base) ID: {eval_proprietary_vs_base.workflow_id}")
print(f"Status: {eval_proprietary_vs_base.status}")

## 🏃‍♂️ Evaluation 2: Proprietary Model vs Fine-tuned Model

Compare the proprietary model's output against the fine-tuned model, with the judge considering alignment with the golden answer.

In [ ]:
finetuned_model = "ivprov/Meta-Llama-3.1-8B-Instruct-Reference-helpsteer3-sft-d5865876-e2abadd3"

In [ ]:
# TODO: Exercise 6 - Create second evaluation (Proprietary vs Fine-tuned)
# This is similar to Exercise 5, but comparing against the fine-tuned model

finetuned_model_config = {
    "model": finetuned_model,
    # TODO: Set model_source - use "dedicated" for fine-tuned models on dedicated endpoints
    #       or "serverless" if using LoRA serverless
    "model_source": None,  # <-- Replace None
    "system_template": generation_system_template,
    "input_template": input_template,
    "max_tokens": MAX_TOKENS,
    "temperature": TEMPERATURE
}

# TODO: Create the evaluation comparing proprietary_model_config vs finetuned_model_config
# Use the same pattern as Exercise 5

eval_proprietary_vs_finetuned = None  # <-- Replace with evaluation.create() call

print(f"Eval 2 (Proprietary vs Fine-tuned) ID: {eval_proprietary_vs_finetuned.workflow_id}")
print(f"Status: {eval_proprietary_vs_finetuned.status}")

## ⏳ Wait for Evaluations to Complete

In [ ]:
# Get status for all evaluations and wait until they have results
while True:
    status_proprietary_vs_base = together_client.evaluation.status(eval_proprietary_vs_base.workflow_id)
    status_proprietary_vs_finetuned = together_client.evaluation.status(eval_proprietary_vs_finetuned.workflow_id)
    
    if status_proprietary_vs_base.results and status_proprietary_vs_finetuned.results:
        break
    
    print("Waiting for evaluations to complete...")
    time.sleep(10)

print("All evaluations completed!")
print(f"Proprietary vs Base: {status_proprietary_vs_base}")
print(f"Proprietary vs Fine-tuned: {status_proprietary_vs_finetuned}")

Now we can take a look at the results

In [ ]:
def print_comparison_summary(status, eval_name, model_a_name, model_b_name):
    """Print a summary of comparison results."""
    if not status.results:
        print(f"{eval_name}: Results not available yet")
        return
    
    results = status.results
    total = results.get('A_wins', 0) + results.get('B_wins', 0) + results.get('Ties', 0)
    if total == 0:
        print(f"{eval_name}: No results yet")
        return
    
    a_wins = results.get('A_wins', 0)
    b_wins = results.get('B_wins', 0)
    ties = results.get('Ties', 0)
    
    print(f"\n{'='*60}")
    print(f"{eval_name}")
    print(f"{'='*60}")
    print(f"Total samples: {total}")
    print(f"{model_a_name} wins: {a_wins} ({a_wins/total*100:.1f}%)")
    print(f"{model_b_name} wins: {b_wins} ({b_wins/total*100:.1f}%)")
    print(f"Ties: {ties} ({ties/total*100:.1f}%)")
    
    if a_wins > b_wins:
        print(f"✅ Winner: {model_a_name}")
    elif b_wins > a_wins:
        print(f"✅ Winner: {model_b_name}")
    else:
        print("🤝 It's a tie!")

# Display results summary using status from previous cell
print_comparison_summary(status_proprietary_vs_base, "Eval 1: Proprietary vs Base", "Proprietary Model", "Base Model")
print_comparison_summary(status_proprietary_vs_finetuned, "Eval 2: Proprietary vs Fine-tuned", "Proprietary Model", "Fine-tuned Model")

## Scores from Evaluation can be used in addition to performance numbers to better decide which model to choose for a particular settings.

### Download Results File
We can download the results file with feedback from the LLM Judge about each decision.

In [ ]:
status_proprietary_vs_finetuned.results['result_file_id']

In [ ]:
eval_proprietary_vs_finetuned.workflow_id

In [ ]:
# Download results from the comparison of proprietary vs fine-tuned model
OUTPUT_FILE_NAME = "./helpsteer3_proprietary_vs_finetuned_results.jsonl"
results_file_id = status_proprietary_vs_finetuned.results['result_file_id']
results_finetuned = together_client.files.retrieve_content(
    results_file_id,
    output=OUTPUT_FILE_NAME
)

In [ ]:
# Read the results file and print columns for the first line
with open(OUTPUT_FILE_NAME, 'r') as f:
    first_line = json.loads(f.readline())
    print("Columns in the results file:")
    for key in first_line.keys():
        print(f"  - {key}")


In [ ]:
# Print specific fields from the first row
print("\n" + "="*80)
print("FIRST ROW DETAILS")
print("="*80)
print("\n\n\n!!!!!!!!!!!!!! Context Formatted:")
print(first_line['context_formatted'])
print("\n\n\n!!!!!!!!!!!!!! MODEL_TO_EVALUATE_OUTPUT_A:")
print(first_line['MODEL_TO_EVALUATE_OUTPUT_A'])
print("\n\n\n!!!!!!!!!!!!!! Golden Answer:")
print(first_line['golden_answer'])
print("\n\n\n!!!!!!!!!!!!!! Judge Feedback (Original Order):")
print(first_line['judge_feedback_original_order'])
